In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "r_star"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [4]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/Bobby/Desktop/Fed Challenge 2025/NEUFed2025/figures')

In [5]:
today = dt.date.today()

hlb_df = pd.read_excel(ut.get_repo_root() / "code" / "data" / "hlb_2.xlsx")

r_star = hlb_df.set_index(pd.to_datetime(hlb_df["Date"]))['Natural Rate (r*)'].rename("HLW Model R* (Quarterly)")

r_star.tail()


Date
2024-04-01    0.857796
2024-07-01    0.819237
2024-10-01    0.825803
2025-01-01    0.792333
2025-04-01    0.839838
Name: HLW Model R* (Quarterly), dtype: float64

In [6]:
today = dt.date.today()

effr = fred.get_series(
    series_id="EFFR",
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Fed Funds")

pce = fred.get_series(
    series_id="PCEPI",
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
    units='pc1'
).rename("PCE")

clv_fed = fred.get_series(
    series_id="REAINTRATREARAT1MO",
    observation_start=dt.date(2019, 1, 1),
    observation_end=today,
).rename("Cleveland Fed Estimated 1 Month Real Interest Rate")

real = (effr - pce).dropna().rename('Fisher Implied Real Rate ')

joined_df = pd.concat(
    [real, r_star, clv_fed],
    axis = 1,
    join='outer'
).fillna(method='ffill')

joined_df = joined_df[joined_df.index >= '2023-01-01']
joined_df


C:\Users\Bobby\AppData\Local\Temp\ipykernel_2236\727116476.py:29: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  ).fillna(method='ffill')


,Fisher Implied Real Rate,HLW Model R* (Quarterly),Cleveland Fed Estimated 1 Month Real Interest Rate
2023-01-01,-1.17773,1.184529,1.817899
2023-02-01,-0.62780,1.184529,1.982413
2023-03-01,0.22173,1.184529,4.151604
2023-04-01,0.35399,1.050807,1.845097
2023-05-01,1.04654,1.050807,1.607673
2023-06-01,1.79548,1.050807,6.004632
2023-07-01,1.71096,0.981644,2.854389
2023-08-01,1.89794,0.981644,3.100313
2023-09-01,1.88111,0.981644,2.492336
2023-10-01,2.30674,0.864953,3.005341


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [21]:
import numpy as np

# First make the figure
fig = go.Figure()


# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'R* and Real Rates Estimates<br><sup>Monthly </sup>'),
    xaxis_title="Date",
    yaxis_title="Change (%)",
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".2f%",
    ticksuffix="%",
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    dtick='M12',
    tickformat='%Y',
    title_text=None,
)

fig.update_layout(
    height=550,
    width=1150,
        legend=dict(
            orientation='v'
        )
)

fig.add_hline(
    y=1,
    line_dash="dash",
    line_color="red",
    annotation_text="Sep. SEP Fisher-Implied R*: 1%</br>",
    annotation_position="bottom left"
)

ut.add_end_labels(fig = fig, offset=0.05)

fig.update_layout(legend=dict(font=dict(size=20)))

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")